In [16]:
#SELECT THE SIZE OF THE NEW SF THAT YOU WANT TO CREATE
SF=3

In [22]:
import os

#SET VARIABLES
PATH=os.getcwd().replace("\\", "/")+'/dags/data/sf_'+str(SF)
print(PATH)

c:/Users/josub/Desktop/BDMA/Brussels/Data_Warehouses/TPC-DI OFFICIAL/tpcdi-benchmark-postgres/src/dags/data/sf_3


In [18]:
import shutil


shutil.copytree('templates\\dags_temp', 'dags\\sf_'+str(SF))
shutil.copy('templates\\plugins_temp\\customermgmt_conversion.py', 'plugins\\customermgmt_conversion_sf'+str(SF)+'.py')
with open('dags\\sf_'+str(SF)+'\\full_historical_load_etl_dag.py', "r") as file:
    content = file.read()
modified_content = content.replace('INPUT_SF', str(SF))
with open('dags\\sf_'+str(SF)+'\\full_historical_load_etl_dag.py', "w") as file:
    file.write(modified_content)
    
with open('plugins\\customermgmt_conversion_sf'+str(SF)+'.py', "r") as file:
    content = file.read()
modified_content = content.replace('INPUT_SF', str(SF))
with open('plugins\\customermgmt_conversion_sf'+str(SF)+'.py', "w") as file:
    file.write(modified_content)

In [23]:
output_file = "dags/sf_"+str(SF)+"/staging_data_commands.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
        sql = f"""COPY staging.batchdate FROM '{PATH}/Batch1/BatchDate.txt';\n
COPY staging.cashtransaction FROM '{PATH}/Batch1/CashTransaction.txt' delimiter '|';\n
COPY staging.dailymarket FROM '{PATH}/Batch1/DailyMarket.txt' delimiter '|';\n
COPY staging.date FROM '{PATH}/Batch1/Date.txt' delimiter '|';\n
COPY staging.holdinghistory FROM '{PATH}/Batch1/HoldingHistory.txt' delimiter '|';\n
COPY staging.hr FROM '{PATH}/Batch1/HR.csv' delimiter ',' CSV;\n
COPY staging.industry FROM '{PATH}/Batch1/Industry.txt' delimiter '|';\n
COPY staging.prospect FROM '{PATH}/Batch1/Prospect.csv' delimiter ',' CSV;\n
COPY staging.statustype FROM '{PATH}/Batch1/StatusType.txt' delimiter '|';\n
COPY staging.taxrate FROM '{PATH}/Batch1/TaxRate.txt' delimiter '|';\n
COPY staging.time FROM '{PATH}/Batch1/Time.txt' delimiter '|';\n
COPY staging.tradehistory FROM '{PATH}/Batch1/TradeHistory.txt' delimiter '|';\n
COPY staging.trade FROM '{PATH}/Batch1/Trade.txt' delimiter '|' null as '';\n
COPY staging.tradetype FROM '{PATH}/Batch1/TradeType.txt' delimiter '|';\n
COPY staging.watchhistory FROM '{PATH}/Batch1/WatchHistory.txt' delimiter '|';\n
COPY staging.audit FROM '{PATH}/Batch1_audit.csv' DELIMITER ',' HEADER CSV NULL AS '';\n       
"""

        file.write(sql)
        

In [24]:
output_file = "dags/sf_"+str(SF)+"/staging_finwire_load1.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
    for i in range(1967,2017):
        for j in range(4):
            sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE{i}Q{j+1}';\n"
            file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q1';\n"
    file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q2';\n"
    file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q3';\n"
    file.write(sql)

In [25]:
output_file = "dags/sf_"+str(SF)+"/load_staging_customermgmt_db.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
    sql = f"COPY staging.customermgmt FROM '{PATH}/Batch1/CustomerMgmt.csv' DELIMITER ',' HEADER CSV NULL AS '';\n"
    file.write(sql)